In [1]:
import numpy as np
from qore import Mine, ASP, QAOA, VQE
from qore.utils import measure_operator, get_bitstring_probabilities

In [2]:
penalty = 10.0
qmine = Mine(np.array([[-2., 3., 1.], [float('inf'), 5., float('inf')]]))

# penalty = 20.0
# qmine = Mine(np.array([[-2., 3., 1., -1.], [float('inf'), 5., 3., float('inf')]]))

# penalty = 5.0
# qmine = Mine(np.array([[-2.0, 3.0, -1.0, -2.0, -1.0], [float('inf'), 5.0, 3.0, 10.0, float('inf')], [float('inf'), float('inf'), 4.0, float('inf'), float('inf')]]))

qmine.plot_mine()

+---+--------+-------+-------+
|   |   0    |   1   |   2   |
+---+--------+-------+-------+
| 0 | -2.000 | 3.000 | 1.000 |
| 1 |  inf   | 5.000 |  inf  |
+---+--------+-------+-------+


In [3]:
qmine.graph

defaultdict(list, {3: [0, 1, 2], 0: [], 1: [], 2: []})

In [4]:
print(qmine.gen_Hamiltonian(penalty=penalty))

-0.0 * IIII
+ 1.0 * IIII
- 1.0 * IIIZ
- 1.5 * IIII
+ 1.5 * IIZI
- 0.5 * IIII
+ 0.5 * IZII
- 2.5 * IIII
+ 2.5 * ZIII
+ 0.0 * IIII
+ 2.5 * IIII
+ 2.5 * IIIZ
- 2.5 * ZIII
- 2.5 * ZIIZ
+ 2.5 * IIII
+ 2.5 * IIZI
- 2.5 * ZIII
- 2.5 * ZIZI
+ 2.5 * IIII
+ 2.5 * IZII
- 2.5 * ZIII
- 2.5 * ZZII


In [6]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit import Aer

algorithm_globals.random_seed = 1953
# backend = Aer.get_backend('statevector_simulator')

def analysis(circ):
    # print("Average Profit:                          ", measure_operator(qmine.Hp, circ, QuantumInstance(backend=Aer.get_backend('statevector_simulator'))))
    # print("Average Number of Smoothness Violations: ", measure_operator(qmine.Hs, circ, QuantumInstance(backend=Aer.get_backend('statevector_simulator'))) / penalty)
    x = get_bitstring_probabilities(circ, QuantumInstance(backend=Aer.get_backend('statevector_simulator')), shots=1024)
    bitstr, prob = max(x.items(), key=lambda item: item[1])
    print(f"The most probable configuration and the corresponding probability: {bitstr, prob}")
    # qmine.plot_mine_state(bitstr)
    # print("Current profit:               ", qmine.get_profit(bitstr))
    # print("Current number of violations: ", qmine.get_violation(bitstr))
    # print(x)
    # print("------------------------------------------------------------")

In [12]:
evol_time = 20
nsteps = 50


circuit = ASP(qmine.gen_Hamiltonian(penalty=penalty), 
    evol_time=evol_time, 
    nsteps=nsteps, 
    callback=analysis, 
    # quantum_instance=QuantumInstance(backend=Aer.get_backend('statevector_simulator'))
).construct_circuit()

analysis(circuit)
# print(circuit.draw())
# print(circuit.decompose().draw())

TypeError: ListOp expecting objects of type OperatorBase, got      ┌───┐┌───┐
q_0: ┤ X ├┤ H ├
     ├───┤├───┤
q_1: ┤ X ├┤ H ├
     ├───┤├───┤
q_2: ┤ X ├┤ H ├
     ├───┤├───┤
q_3: ┤ X ├┤ H ├
     └───┘└───┘

In [ ]:
from qiskit.algorithms.optimizers import COBYL

p = 3
qaoa = QAOA(
        optimizer=COBYLA(),
        quantum_instance=QuantumInstance(
            backend=Aer.get_backend("statevector_simulator")
        ),
        reps=p,
    )
qaoa.compute_minimum_eigenvalue(qmine.gen_Hamiltonian(penalty=penalty))
circuit = qaoa.get_optimal_circuit()
analysis(circuit)